<a href="https://colab.research.google.com/github/Praveen-Prabhu/code-generation/blob/main/code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### HuggingFace & W&B Authentication

In [22]:
from huggingface_hub import login, HfApi, HfFolder
import wandb
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_TOKEN = userdata.get('WANDB_TOKEN')

login(token=HF_TOKEN)
wandb.login(key=WANDB_TOKEN)

wandb.init(
    project="codegen-llama-3.1-8b-grpo",
    job_type="training",
    anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [23]:
# Define Hugging Face Repository for Saving Checkpoints
HF_REPO = "PraveenPrabhu/codegen-llama-3.1-8b-grpo"

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [5]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-12 10:07:49 [__init__.py:239] Automatically detected platform cuda.


Load up `Llama 3.1 8B Instruct`, and set parameters

In [6]:
# !export VLLM_USE_V1=0

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.59 GB. Also swap space = 5 GB.
WARNING 04-12 10:08:02 [config.py:2704] Casting torch.bfloat16 to torch.float16.
INFO 04-12 10:08:15 [config.py:600] This model supports multiple tasks: {'score', 'reward', 'classify',

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 04-12 10:09:46 [model_runner.py:1598] Graph capturing finished in 47 secs, took 0.53 GiB
INFO 04-12 10:09:46 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 59.67 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [8]:
import re
from datasets import load_dataset, Dataset
import subprocess

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

# def extract_xml_answer(text: str) -> str:
#     answer = text.split("<answer>")[-1]
#     answer = answer.split("</answer>")[0]
#     return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# # uncomment middle messages for 1-shot prompting
# def get_gsm8k_questions(split = "train") -> Dataset:
#     data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
#     data = data.map(lambda x: { # type: ignore
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': extract_hash_answer(x['answer'])
#     }) # type: ignore
#     return data # type: ignore

# dataset = get_gsm8k_questions()

# Load and map HumanEval dataset
def get_humaneval_questions(split = "test") -> Dataset:
    data = load_dataset('openai_humaneval')[split]

    # Process each sample
    def process_sample(x):
        prompt = x['prompt']
        canonical_solution = x['canonical_solution']
        test_code = x['test']

        # Check if test code is valid
        if isinstance(test_code, str) and len(test_code) > 10:
            return {
                'prompt': [
                    {'role': 'system', 'content': SYSTEM_PROMPT},
                    {'role': 'user', 'content': prompt}
                ],
                'answer': canonical_solution,
                'test': test_code
            }
        else:
            print(f"Warning: Invalid test code for task: {prompt[:50]}...")
            return None

    processed_data = data.map(process_sample)
    # Filter out None values
    processed_data = processed_data.filter(lambda x: x is not None)

    return processed_data

dataset = get_humaneval_questions()
test_code_list = dataset["test"]

# Reward functions
# def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
#     responses = [completion[0]['content'] for completion in completions]
#     q = prompts[0][-1]['content']
#     extracted_responses = [extract_xml_answer(r) for r in responses]
#     print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
#     return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

# def int_reward_func(completions, **kwargs) -> list[float]:
#     responses = [completion[0]['content'] for completion in completions]
#     extracted_responses = [extract_xml_answer(r) for r in responses]
#     return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

# Function to run generated code and capture output from assertions
def run_generated_code(code, test_code):
    try:
        # Clean up the test code if it's malformed
        if isinstance(test_code, list) or isinstance(test_code, str) and "," in test_code and len(test_code) < 100:
            print("Warning: Test code appears to be malformed")
            return 0.0

        # Combine the generated solution with the test code
        full_code = f"{code}\n\n{test_code}"

        # Print for debugging
        print(f"Running code:\n{full_code[:200]}...")

        # Create a temporary file to execute
        with open("temp_test.py", "w") as f:
            f.write(full_code)

        # Run the test file
        result = subprocess.run(
            ["python3", "temp_test.py"],
            capture_output=True, text=True, timeout=15
        )

        # Check if execution was successful (no errors)
        if result.returncode == 0:
            print("Test passed successfully!")
            return 2.0  # Increased reward for correct execution
        else:
            print(f"Test failed with error: {result.stderr}")
            return 0.0  # Test failed
    except subprocess.TimeoutExpired:
        print("Execution timed out")
        return -0.5  # Penalize timeouts more severely
    except Exception as e:
        print(f"Execution error: {str(e)}")
        return 0.0  # Test failed

# #   Function to Verify Output with Z3 SMT Solver
# def verify_with_smt_solver(output, constraints):
#     solver = Solver()
#     z_output = Int("output")
#     solver.add(z_output == output)

#     for constraint in constraints:
#         solver.add(eval(constraint))  # Convert string to Z3 expressions

#     return solver.check() == sat

def extract_answer_code(text: str) -> str:
    # Try to extract code from the <answer> tags
    match = re.search(r"<answer>\s*(.*?)\s*</answer>", text, re.DOTALL)
    if match:
        return match.group(1).strip()

    # Try to extract code from triple backticks if it's not in <answer> tags
    match = re.search(r"```(?:python)?\s*(.*?)\s*```", text, re.DOTALL)
    if match:
        return match.group(1).strip()

    # Return the original text if no patterns match
    return text.strip()

# Example reward function with HumanEval tests
def make_rl_reward_function(test_code_list):
    def rl_reward_function(prompts, completions, **kwargs):
        rewards = []

        for i, completion in enumerate(completions):
            try:
                if i >= len(test_code_list):
                    print(f"Warning: No test code available for sample {i}")
                    rewards.append(0.0)
                    continue

                idx = i  # Use the correct index
                test_code = test_code_list[idx]

                # Skip if test code is problematic
                if not isinstance(test_code, str) or len(test_code) < 10:
                    print(f"Warning: Test code for sample {i} is invalid")
                    rewards.append(0.0)
                    continue

                # Extract code from completion
                generated_code = extract_answer_code(completion[0]["content"])

                # Add debugging print
                print(f"Sample {i} - Generated code (first 100 chars): {generated_code[:100]}")

                # Run the code and get reward
                reward = run_generated_code(generated_code, test_code)
                rewards.append(reward)

                print(f"Sample {i}: Reward = {reward}")
            except Exception as e:
                print(f"Error processing completion {i}: {str(e)}")
                rewards.append(0.0)

        return rewards
    return rl_reward_function


README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

Filter:   0%|          | 0/164 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [27]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1, # Set to 1 for a full training run
    # max_steps = 1,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "checkpoints",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [28]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        make_rl_reward_function(test_code_list),
    ],
    args = training_args,
    train_dataset = dataset,
)
# trainer.train()

In [30]:
# Load Checkpoint from Hugging Face if Exists
from huggingface_hub import hf_hub_download
import shutil

def download_checkpoint(repo_id, filename, local_dir="hf_checkpoints"):
    """Download a checkpoint file from Hugging Face and return its local path."""
    os.makedirs(local_dir, exist_ok=True)
    try:
        return hf_hub_download(repo_id, filename, local_dir=local_dir)
    except:
        return None  # Return None if checkpoint does not exist

lora_path = download_checkpoint(HF_REPO, "lora_adapter.safetensors")
optimizer_path = download_checkpoint(HF_REPO, "optimizer_state.pth")
trainer_path = download_checkpoint(HF_REPO, "trainer_state.pth")
tokenizer_path = download_checkpoint(HF_REPO, "tokenizer.json")

if lora_path and optimizer_path and trainer_path and tokenizer_path:
    model.load_lora(lora_path)
    trainer.load_state_dict(torch.load(trainer_path))
    trainer.optimizer.load_state_dict(torch.load(optimizer_path))
    tokenizer = torch.load(tokenizer_path)
    print("Resumed training from Hugging Face checkpoint!")

In [31]:
# Train Model
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 164 | Num Epochs = 1 | Total steps = 164
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


Sample 0 - Generated code (first 100 chars): def below_zero(operations: List[int]) -> bool:
    """
    This function checks if the balance of a 
Running code:
def below_zero(operations: List[int]) -> bool:
    """
    This function checks if the balance of a bank account falls below zero after a series of deposit and withdrawal operations.

    Args:
      ...
Test failed with error: Traceback (most recent call last):
  File "/content/temp_test.py", line 1, in <module>
    def below_zero(operations: List[int]) -> bool:
                               ^^^^
NameError: name 'List' is not defined. Did you mean: 'list'?

Sample 0: Reward = 0.0
Sample 1 - Generated code (first 100 chars): from typing import List

def below_zero(operations: List[int]) -> bool:
    """
    Detects if a ban
Running code:
from typing import List

def below_zero(operations: List[int]) -> bool:
    """
    Detects if a bank account balance falls below zero after a series of deposits and withdrawals.
    
    Args:

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / rl_reward_function
1,-0.000000,1.375000,1.069463,247.500000,0.000000,0.041667,0.000000,0.000000,1.333333
2,0.000000,1.666667,0.816497,306.333344,0.000000,0.000000,0.000000,0.000000,1.666667
3,0.000000,1.708333,0.842862,611.833374,0.000004,0.041667,0.000000,0.000000,1.666667
4,0.000000,1.316667,0.798796,238.000000,0.000008,-0.350000,0.000000,0.000000,1.666667
5,0.000000,2.000000,0.000000,409.833344,0.000005,0.000000,0.000000,0.000000,2.000000
6,0.000000,0.750167,1.078174,149.000000,0.000014,-0.249833,0.000000,0.000000,1.000000
7,0.000000,1.666667,0.816497,324.333344,0.000004,0.000000,0.000000,0.000000,1.666667
8,0.000000,1.541667,0.687265,354.166687,0.000004,-0.458333,0.000000,0.000000,2.000000
9,0.000000,1.253667,1.166054,450.833344,0.000006,-0.079667,0.000000,0.000000,1.333333
10,0.000000,0.573000,0.916440,154.666672,0.000059,-0.093667,0.000000,0.000000,0.666667


Streaming output truncated to the last 5000 lines.
Test passed successfully!
Sample 2: Reward = 2.0
Sample 3 - Generated code (first 100 chars): def eat(number, need, remaining):
    """
    Calculate the number of carrots eaten and the number o
Running code:
def eat(number, need, remaining):
    """
    Calculate the number of carrots eaten and the number of carrots left after meals.

    Args:
    number (int): The number of carrots already eaten.
    ne...
Test passed successfully!
Sample 3: Reward = 2.0
Sample 4 - Generated code (first 100 chars): def eat(number, need, remaining):
    """
    Calculate the total number of carrots eaten and the nu
Running code:
def eat(number, need, remaining):
    """
    Calculate the total number of carrots eaten and the number left after meals.
    
    Args:
        number (int): The number of carrots you have eaten.
  ...
Test passed successfully!
Sample 4: Reward = 2.0
Sample 5 - Generated code (first 100 chars): def eat(number: int, need: in

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / rl_reward_function
1,-0.000000,1.375000,1.069463,247.500000,0.000000,0.041667,0.000000,0.000000,1.333333
2,0.000000,1.666667,0.816497,306.333344,0.000000,0.000000,0.000000,0.000000,1.666667
3,0.000000,1.708333,0.842862,611.833374,0.000004,0.041667,0.000000,0.000000,1.666667
4,0.000000,1.316667,0.798796,238.000000,0.000008,-0.350000,0.000000,0.000000,1.666667
5,0.000000,2.000000,0.000000,409.833344,0.000005,0.000000,0.000000,0.000000,2.000000
6,0.000000,0.750167,1.078174,149.000000,0.000014,-0.249833,0.000000,0.000000,1.000000
7,0.000000,1.666667,0.816497,324.333344,0.000004,0.000000,0.000000,0.000000,1.666667
8,0.000000,1.541667,0.687265,354.166687,0.000004,-0.458333,0.000000,0.000000,2.000000
9,0.000000,1.253667,1.166054,450.833344,0.000006,-0.079667,0.000000,0.000000,1.333333
10,0.000000,0.573000,0.916440,154.666672,0.000059,-0.093667,0.000000,0.000000,0.666667


Sample 0 - Generated code (first 100 chars): def fibfib(n: int) -> int:
    """
    This function calculates the n-th element of the fibfib numbe
Running code:
def fibfib(n: int) -> int:
    """
    This function calculates the n-th element of the fibfib number sequence.
    
    The fibfib sequence is a sequence similar to the Fibonacci sequence, 
    defin...
Test passed successfully!
Sample 0: Reward = 2.0
Sample 1 - Generated code (first 100 chars): 
Running code:




METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate('(()()) ((())) () ((())()())') == [
        '(()())', '((()))', '()', '((())()())'
    ]
    a...
Test passed successfully!
Sample 1: Reward = 2.0
Sample 2 - Generated code (first 100 chars): def fibfib(n: int, memo={}):
    """
    Compute the nth element of the FibFib sequence.

    The Fi
Running code:
def fibfib(n: int, memo={}):
    """
    Compute the nth element of the FibFib sequence.

    The FibFib sequence 

TrainOutput(global_step=164, training_loss=0.00011906859206104648, metrics={'train_runtime': 16009.0404, 'train_samples_per_second': 0.01, 'train_steps_per_second': 0.01, 'total_flos': 0.0, 'train_loss': 0.00011906859206104648})

In [32]:
# Save All States After Training
# torch.save(trainer.state_dict(), "trainer_state.pth")
model.save_pretrained_merged("lora_adapter", tokenizer, save_method = "lora",)
torch.save(tokenizer, "tokenizer.json")
torch.save(trainer.optimizer.state_dict(), "optimizer_state.pth")

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... Done.


In [33]:
# Finish W&B Logging
wandb.finish()

train/completion_length,▄▃▅▆▃▃▄▆▂▃▄▂▂▃▆▅▇▂▁▅▃▄▃▃▆▅▃▄▆▅█▆▃▄▆▃▄▄▂▅
train/epoch,▁▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇████
train/grad_norm,▁▃▂▁▁▂▁▂▁▂▁▂▂▂▂▂▂▁▃▂▁▁▁▁▃▁▂▂▁▁▃▁▁▁▂█▂▂▁▂
train/kl,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▁▃▁▃▃▄▂▇▃▄▆▄▅▅▃▃▃▁▂▃▄▂▃█
train/learning_rate,▃▄▆▇█████▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
train/loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▁▅▃▂▅▁▃▃▅▁▂▂▅▂▃▂▁▆█▆
train/reward,█▇▅█▁▄▆▇██▇▅▆█▇▆█▇█▆██▆█▄█▇████▄▅██▇█▆▇▇
train/reward_std,▅▅▁▆▆▁▅▇▂▄▁▁▆▂▇█▁▁▁▆█▆▆▆▆▆▁▁▂▂▁▅▆▆▆▆▂▃▆▆
train/rewards/rl_reward_function,▆▇▇▅▃▆▁▇█▆▅▇▇▆█▇███▇▇▇█▇███▇████▅▆▇▇▆▇██
train/rewards/soft_format_reward_func,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [34]:
# Upload All States to Hugging Face for Persistence
api = HfApi()

# api.upload_file(
#     path_or_fileobj="lora_adapter.safetensors",
#     path_in_repo="lora_adapter.safetensors",
#     repo_id=HF_REPO,
#     token=HF_TOKEN,
# )

model.push_to_hub_merged(HF_REPO, tokenizer, save_method = "lora")

# api.upload_file(
#     path_or_fileobj="trainer_state.pth",
#     path_in_repo="trainer_state.pth",
#     repo_id=HF_REPO,
#     token=HF_TOKEN,
# )

api.upload_file(
    path_or_fileobj="optimizer_state.pth",
    path_in_repo="optimizer_state.pth",
    repo_id=HF_REPO,
    token=HF_TOKEN,
)

api.upload_file(
    path_or_fileobj="tokenizer.json",
    path_in_repo="tokenizer.json",
    repo_id=HF_REPO,
    token=HF_TOKEN,
)

Unsloth: Saving LoRA adapters. Please wait...


adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/PraveenPrabhu/codegen-llama-3.1-8b-grpo


optimizer_state.pth:   0%|          | 0.00/171M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PraveenPrabhu/codegen-llama-3.1-8b-grpo/commit/18370e9f77ed5071b1e8def41b29a0aacdef405c', commit_message='Upload tokenizer.json with huggingface_hub', commit_description='', oid='18370e9f77ed5071b1e8def41b29a0aacdef405c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/PraveenPrabhu/codegen-llama-3.1-8b-grpo', endpoint='https://huggingface.co', repo_type='model', repo_id='PraveenPrabhu/codegen-llama-3.1-8b-grpo'), pr_revision=None, pr_num=None)

<a name="Inference"></a>
### Inference

Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [35]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "write a python program to calculate first n numbers of fibonacci series"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:20<00:00, 20.42s/it, est. speed input: 2.35 toks/s, output: 17.73 toks/s]


'**Fibonacci Series Calculator**\n================================\n\nBelow is a Python program that calculates the first `n` numbers of the Fibonacci series:\n\n```python\ndef fibonacci(n):\n    """\n    Calculates the first n numbers of the Fibonacci series.\n\n    Args:\n        n (int): The number of Fibonacci numbers to generate.\n\n    Returns:\n        list: A list of the first n Fibonacci numbers.\n    """\n    fib_series = [0, 1]\n    while len(fib_series) < n:\n        fib_series.append(fib_series[-1] + fib_series[-2])\n    return fib_series[:n]\n\ndef main():\n    n = int(input("Enter the number of Fibonacci numbers to generate: "))\n    if n <= 0:\n        print("Please enter a positive integer.")\n    else:\n        print(fibonacci(n))\n\nif __name__ == "__main__":\n    main()\n```\n\n**How to use:**\n\n1. Run the program.\n2. Enter the number of Fibonacci numbers you want to generate.\n3. The program will print the first `n` numbers of the Fibonacci series.\n\n**Explanati

In [ ]:
print(output)

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [36]:
# model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [37]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "write a python program to calculate first n numbers of fibonacci series"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("lora_adapter"),
)[0].outputs[0].text

Processed prompts: 100%|██████████| 1/1 [00:25<00:00, 25.83s/it, est. speed input: 2.71 toks/s, output: 16.22 toks/s]


'**Fibonacci Series Calculator**\n================================\n\nBelow is a Python program that calculates the first n numbers of the Fibonacci series.\n\n### Code\n\n```python\ndef fibonacci(n):\n    """\n    Calculate the first n numbers of the Fibonacci series.\n\n    Args:\n        n (int): The number of Fibonacci numbers to generate.\n\n    Returns:\n        list: A list of the first n Fibonacci numbers.\n    """\n    fib_series = [0, 1]\n    while len(fib_series) < n:\n        fib_series.append(fib_series[-1] + fib_series[-2])\n    return fib_series[:n]\n\ndef main():\n    n = int(input("Enter the number of Fibonacci numbers to generate: "))\n    print("Fibonacci series:", fibonacci(n))\n\nif __name__ == "__main__":\n    main()\n```\n\n### Explanation\n\n1. The `fibonacci` function initializes a list `fib_series` with the first two Fibonacci numbers (0 and 1). It then enters a loop that continues until the length of `fib_series` reaches the desired value of `n`.\n2. In each 

In [38]:
print(output)

**Fibonacci Series Calculator**

Below is a Python program that calculates the first n numbers of the Fibonacci series.

### Code

```python
def fibonacci(n):
    """
    Calculate the first n numbers of the Fibonacci series.

    Args:
        n (int): The number of Fibonacci numbers to generate.

    Returns:
        list: A list of the first n Fibonacci numbers.
    """
    fib_series = [0, 1]
    while len(fib_series) < n:
        fib_series.append(fib_series[-1] + fib_series[-2])
    return fib_series[:n]

def main():
    n = int(input("Enter the number of Fibonacci numbers to generate: "))
    print("Fibonacci series:", fibonacci(n))

if __name__ == "__main__":
    main()
```

### Explanation

1. The `fibonacci` function initializes a list `fib_series` with the first two Fibonacci numbers (0 and 1). It then enters a loop that continues until the length of `fib_series` reaches the desired value of `n`.
2. In each iteration of the loop, the next Fibonacci number is calculated by ad

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [39]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [40]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)